# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules and define a couple of useful functions for visualization:

In [1]:
from copy import deepcopy
import os
import multiprocessing as mp

import gsd.hoomd
import mbuild as mb
import numpy as np
import gsd.pygsd
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict
#had the below because I didnt know this function was in mobility_kmc
#from cmeutils.gsd_utils import snap_molecule_cluster
from morphct.mobility_kmc import snap_molecule_indices
def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp

/Users/jamesrushing/opt/miniconda3/envs/morph/lib/python3.7/site-packages/pyscf/lib/misc.py:50: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
gsd_file = "/Users/jamesrushing/cmelab/data/workspace/4700e7da2f41cc6c77e08977f1f8d94c/trajectory.gsd"
g = gsd.pygsd.GSDFile(open(gsd_file, "rb"))
trajectory = gsd.hoomd.HOOMDTrajectory(g)
gsd_length= len(trajectory)
with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    start_snap = f[1]
    end_snap = f[gsd_length - 1]

    
box = start_snap.configuration.box[:3]
ref_distance = 3.563594872561358
unwrapped_positions = start_snap.particles.position + start_snap.particles.image * box
start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
unwrap_snap = deepcopy(start_snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [amber_dict[i].symbol for i in start_snap.particles.types]
comp = from_snapshot(unwrap_snap, scale=0.1*ref_distance)
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
chromo_ids = np.array([0,1,2,4,6,7,10,11,12,13,19,20,22,23,24,25,27,28,29,30,31,32,91,92,93,94,97,98,99,100,101,102,161,162,163,165,166,168,169,170,171,172,178,179,180,181,177,175,176,174,173,17,15,18,16,14,13])
for i,p in enumerate(comp.particles()):
    if i in chromo_ids:
        p.name = "Kr"
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

below i want to get the  lumos for just that back bone.

below im going to sort out the linear algebra of indexing chromophores given the molecule index and the chomo_ids obtained above. we have 186 atoms per molecule. 

In [4]:
gsd_file = "/Users/jamesrushing/cmelab/data/workspace/0769578cc7d05faca991a8089c23e0e0/trajectory.gsd"
g = gsd.pygsd.GSDFile(open(gsd_file, "rb"))
trajectory = gsd.hoomd.HOOMDTrajectory(g)
gsd_length= len(trajectory)
with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    start_snap = f[1]
    end_snap = f[gsd_length - 1]

gsd_mol_index = snap_molecule_indices(start_snap)

In [5]:
gsd_mol_index = snap_molecule_indices(start_snap)
k = np.count_nonzero(gsd_mol_index==0)
print(k)

186


The following function extrapolates the manually prescribes indeces )chromo_ids from the first molocule to however many molecules there are. 

In [6]:
gsd_mol_index = snap_molecule_indices(start_snap)
k = np.count_nonzero(gsd_mol_index==0)
# k is the number of atoms per molecule as given by the number of 0's in the molecule index
# given by the snap_molecule_indeces funtion from cme utils. 
master_list = []
sublist = chromo_ids
for i in range(len(np.unique(gsd_mol_index))):         
    master_list.append(sublist)
    sublist = [x + k for x in sublist]

below shows tolal atoms if we ever need

In [7]:
total_index = len(np.unique(gsd_mol_index))*np.count_nonzero(gsd_mol_index==0)
print(total_index)

4092


below shows number of molecules given by the number of unique indices in array given
by gsd_mol_index = snap_molecule_indices(start_snap)

In [8]:
molecules = np.unique(gsd_mol_index)

In [9]:
test_ids = np.concatenate(master_list, axis=0)
print(test_ids)

[   0    1    2 ... 3922 3920 3919]


In [10]:

for i,p in enumerate(comp.particles()):
    if i in test_ids:
        p.name = "Kr"
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [11]:
with gsd.hoomd.open(name='/Users/jamesrushing/cmelab/data/workspace/0769578cc7d05faca991a8089c23e0e0/trajectory.gsd', mode='rb') as f:
    snap = f[0]
    
box = snap.configuration.box[:3]
ref_distance = 3.563594872561358
unwrapped_positions = snap.particles.position + snap.particles.image * box

unwrap_snap = deepcopy(snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [amber_dict[i].symbol for i in snap.particles.types]
comp = from_snapshot(unwrap_snap, scale=0.1*ref_distance)
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [12]:
for i,p in enumerate(comp.particles()):
    if i in test_ids:
        p.name = "Kr"
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol